In [1]:
# custom libraries
import c_utils
import InstrumentDict
import EntityExtract as ee
import WordReader as wr
import CodeGenerator as cg

# imports
import pandas as pd
import time

import json
import os
import ipfsApi

from hedera import (
    Hbar,
    FileCreateTransaction,
    ContractCreateTransaction,
    ContractCallQuery,
    ContractDeleteTransaction,
    AccountBalanceQuery,
    TokenCreateTransaction,
    FileAppendTransaction,
    ContractFunctionParameters,
    TokenInfoQuery,
    TokenUpdateTransaction,
    ContractExecuteTransaction,
    AccountId,
    PrivateKey,
    Client
)

In [2]:
from users import admin_id, admin_key, test_id, test_key

In [3]:
def swap_to_user(user):
    if user == 'admin':
        os.environ['OPERATOR_ID'] = admin_id
        os.environ['OPERATOR_KEY'] = admin_key
    elif user == 'test':
        os.environ['OPERATOR_ID'] = test_id
        os.environ['OPERATOR_KEY'] = test_key
    else:
        pass

    OPERATOR_ID = AccountId.fromString(os.environ["OPERATOR_ID"])
    OPERATOR_KEY = PrivateKey.fromString(os.environ["OPERATOR_KEY"])

    client = Client.forTestnet()
    client.setOperator(OPERATOR_ID, OPERATOR_KEY);
    
    return OPERATOR_ID, OPERATOR_KEY, client

# Full Deployment Demo

### 1) Document Upload & Parameter Extraction

In [4]:
filepath = './files/word/Term Sheet - Demo 2.1.docx'
params, links, instr = wr.parse_fields(filepath)

In [5]:
params

{'tokenName': 'UniFi Token', 'tokenSymbol': 'UNI', 'multiple': 2}

In [6]:
links

{'tokenName': ('UniFi Token', 'UniFi Token'),
 'tokenSymbol': ('UNI', 'UNI'),
 'multiple': ('The principal amount deposited shall be doubled upon payout from the contract owner.',
  2)}

### 2) Publish Document to IPFS

In [7]:
parampath = r'./params.json'
with open(parampath, 'w') as json_file:
    json.dump(params, json_file)

linkpath = r'./links.json'
with open(linkpath, 'w') as json_file:
    json.dump(links, json_file)

In [8]:
api = ipfsApi.Client('127.0.0.1', 5001)

In [9]:
docHash_dict = api.add(filepath)
docHash_dict[0]

{'Name': 'files/word/Term Sheet - Demo 2.1.docx',
 'Hash': 'QmPF5HyxE412164bsiK8YKJekP8apMZAvUDBedt8CPPNzV',
 'Size': '17852'}

In [10]:
paramHash_dict = api.add(parampath)
paramHash_dict

{'Name': 'params.json',
 'Hash': 'QmU3ukKNNKV6kGoXAUubtW2Ps7Ua7qViT3qbqUGpjfAtui',
 'Size': '73'}

In [11]:
linkHash_dict = api.add(linkpath)
linkHash_dict

{'Name': 'links.json',
 'Hash': 'QmS7zTqKhDTuQMKdjuTHNgoMYaYmLUrT158p28syyu6GAV',
 'Size': '192'}

In [12]:
docHash = docHash_dict[0]['Hash']
paramHash = paramHash_dict['Hash']
linkHash = linkHash_dict['Hash']

### 3) Token Creation

In [13]:
OPERATOR_ID, OPERATOR_KEY, client = swap_to_user('admin')

#### Create Token Using Parameters

In [14]:
treasuryId = OPERATOR_ID
treasuryKey = OPERATOR_KEY

tokenCreateTx = (TokenCreateTransaction().setTokenName(params['tokenName'])
                                         .setTokenSymbol(params['tokenSymbol'])
                                         .setDecimals(0)
                                         .setInitialSupply(10000)
                                         .setTreasuryAccountId(treasuryId)
                                         .setAdminKey(treasuryKey)
                                         .setSupplyKey(treasuryKey)
                                         .freezeWith(client)
                                         .sign(treasuryKey))

tokenCreateSubmit = tokenCreateTx.execute(client);
tokenCreateRx = tokenCreateSubmit.getReceipt(client);
tokenId = tokenCreateRx.tokenId;
tokenAddressSol = tokenId.toSolidityAddress();

print("New token ID is " + tokenId.toString())
print(f"New Token Supply: {TokenInfoQuery().setTokenId(tokenId).execute(client).totalSupply}")

New token ID is 0.0.34819504
New Token Supply: 10000


## 4) Generate Ricardian Contract Template

In [15]:
# contract = cg.RicardianContract(fields_demo, instr, filehash)
# contract.generate('./contracts_RicardianContract.sol')

solcjs --bin ./contracts/RicardianContract.sol

In [16]:
with open(r"./contracts_RicardianContract_sol_RicardianContract.bin", encoding='UTF8') as f:
    contents = f.read()

In [17]:
fileCreateTx = FileCreateTransaction().setKeys(OPERATOR_KEY)
fileSubmit = fileCreateTx.execute(client)
fileCreateRx = fileSubmit.getReceipt(client)
bytecodeFileId = fileCreateRx.fileId
print(f'Smart Contract Bytecode File ID: {bytecodeFileId.toString()}')

fileAppendTx = (FileAppendTransaction().setFileId(bytecodeFileId)
                                       .setContents(contents)
                                       .setMaxChunks(10)
                                       .setMaxTransactionFee(Hbar(2)))
fileAppendSubmit = fileAppendTx.execute(client)
fileAppendRx = fileAppendSubmit.getReceipt(client)
print(f'Content added: {fileAppendRx.status.toString()}')

Smart Contract Bytecode File ID: 0.0.34819505
Content added: SUCCESS


### 5) Instantiate Ricardian Contract on Hedera

In [18]:
contractInstantiateTx = (ContractCreateTransaction()
                         .setBytecodeFileId(bytecodeFileId)
                         .setGas(3000000)
                         .setConstructorParameters(ContractFunctionParameters()
                                                   .addAddress(tokenAddressSol)
                                                   .addInt64(params['multiple'])
                                                   .addString(docHash)
                                                   .addString(paramHash)
                                                   .addString(linkHash)
                                                  ))
contractInstantiateSubmit = contractInstantiateTx.execute(client)
contractInstantiateRx = contractInstantiateSubmit.getReceipt(client)
contractId = contractInstantiateRx.contractId
contractAddress = contractId.toSolidityAddress()

print(f'Smart Contract ID: {contractId.toString()}')
print(f'Smart Contract ID in Solidity Format: {contractAddress}')

Smart Contract ID: 0.0.34819506
Smart Contract ID in Solidity Format: 0000000000000000000000000000000002134db2


#### Set Ricardian Contract as Supply Key for Token

In [19]:
tokenInfo2p1 = TokenInfoQuery().setTokenId(tokenId).execute(client)
print(f'Original token supply key: {tokenInfo2p1.supplyKey.toString()}')

tokenUpdateTx = (TokenUpdateTransaction()
                       .setTokenId(tokenId)
                       .setSupplyKey(contractId))

tokenUpdateSubmit = tokenUpdateTx.execute(client)
tokenUpdateRx = tokenUpdateSubmit.getReceipt(client)

tokenInfo2p2 = TokenInfoQuery().setTokenId(tokenId).execute(client)
print(f'New token supply key: {tokenInfo2p2.supplyKey.toString()}')

Original token supply key: 302a300506032b657003210039658e94fdf535fdd0cfef44915376bd2ddedf37c910db55aa6b775803826d28
New token supply key: 0.0.34819506


## 6) Ready to Use!
#### Query: Get Token Address

In [20]:
query_result = (ContractCallQuery()
                .setGas(300000)
                .setContractId(contractId)
                .setFunction("getAddress")
                .setQueryPayment(Hbar(1))
                .execute(client))
message = query_result.getAddress(0)
print("Token Address: ", message)

Token Address:  0000000000000000000000000000000002134db0


#### Query: Treasury Token Balance

In [21]:
tB_str = c_utils.java_to_json(AccountBalanceQuery().setAccountId(treasuryId).execute(client).tokens.toString())
tB = json.loads(tB_str)[tokenId.toString()]

print(f'Treasury Balance: {tB}')

Treasury Balance: 10000


## 7) Configuring Test Account for Eric

In [22]:
testId = AccountId.fromString(test_id)
testKey = PrivateKey.fromString(test_key)

#### Associate Token with Eric's Account

In [23]:
contractExecTx1 = (ContractExecuteTransaction()
                   .setContractId(contractId)
                   .setGas(3000000)
                   .setFunction("tokenAssociate", ContractFunctionParameters().addAddress(testId.toSolidityAddress()))
                   .setMaxTransactionFee(Hbar(2))
                   .freezeWith(client))
contractExecSign1 = contractExecTx1.sign(testKey);
contractExecSubmit1 = contractExecSign1.execute(client);
contractExecRx1 = contractExecSubmit1.getReceipt(client);

print(f'Token association with Test Account: {contractExecRx1.status.toString()}')

Token association with Test Account: SUCCESS


#### Gift 100 tokens to the Eric

In [24]:
contractExecTx2 = (ContractExecuteTransaction()
                   .setContractId(contractId)
                   .setGas(3000000)
                   .setFunction("gift", ContractFunctionParameters()
                                .addAddress(testId.toSolidityAddress())
                                .addInt64(100))
                   .setMaxTransactionFee(Hbar(5))
                   .freezeWith(client))
contractExecSign2 = contractExecTx2.sign(treasuryKey)
contractExecSubmit2 = contractExecSign2.execute(client)
contractExecRx2 = contractExecSubmit2.getReceipt(client)

print(f'Token Transfer from Treasury to Test Account: {contractExecRx2.status.toString()}')

tB_str = c_utils.java_to_json(AccountBalanceQuery().setAccountId(treasuryId).execute(client).tokens.toString())
tB = json.loads(tB_str)[tokenId.toString()]

aB_str = c_utils.java_to_json(AccountBalanceQuery().setAccountId(testId).execute(client).tokens.toString())
aB = json.loads(aB_str)[tokenId.toString()]

print(f'Treasury Balance: {tB}')
print(f'Eric\'s Balance: {aB}')

Token Transfer from Treasury to Test Account: SUCCESS
Treasury Balance: 9900
Eric's Balance: 100


## 8) Fuctionality Demo
#### Swap to Eric's Account

In [25]:
OPERATOR_ID, OPERATOR_KEY, client = swap_to_user('test')

#### Get Document Hash

In [26]:
query_result = (ContractCallQuery()
                .setGas(300000)
                .setContractId(contractId)
                .setFunction("getDocHash")
                .setQueryPayment(Hbar(1))
                .execute(client))

message = query_result.getString(0)
print("Document Hash: ", message)

Document Hash:  QmPF5HyxE412164bsiK8YKJekP8apMZAvUDBedt8CPPNzV


#### Get Parameter Hash

In [27]:
query_result = (ContractCallQuery()
                .setGas(300000)
                .setContractId(contractId)
                .setFunction("getParamHash")
                .setQueryPayment(Hbar(1))
                .execute(client))

message = query_result.getString(0)
print("Param Hash: ", message)

Param Hash:  QmU3ukKNNKV6kGoXAUubtW2Ps7Ua7qViT3qbqUGpjfAtui


#### Get Link Hash

In [28]:
query_result = (ContractCallQuery()
                .setGas(300000)
                .setContractId(contractId)
                .setFunction("getLinkHash")
                .setQueryPayment(Hbar(1))
                .execute(client))

message = query_result.getString(0)
print("Link Hash: ", message)

Link Hash:  QmS7zTqKhDTuQMKdjuTHNgoMYaYmLUrT158p28syyu6GAV


#### Deposit 100 Tokens into the Instrument

In [29]:
deposit_amt = 100

contractExecTx2 = (ContractExecuteTransaction()
                   .setContractId(contractId)
                   .setGas(3000000)
                   .setFunction("deposit", ContractFunctionParameters()
                                .addAddress(testId.toSolidityAddress())
                                .addInt64(deposit_amt))
                   .setMaxTransactionFee(Hbar(5))
                   .freezeWith(client))
contractExecSign2 = contractExecTx2.sign(testKey)
contractExecSubmit2 = contractExecSign2.execute(client)
contractExecRx2 = contractExecSubmit2.getReceipt(client)

print(f'Deposit Tokens from Test Account to Treasury: {contractExecRx2.status.toString()}')
print(f'Deposit Amount: {deposit_amt}')

tB_str = c_utils.java_to_json(AccountBalanceQuery().setAccountId(treasuryId).execute(client).tokens.toString())
tB = json.loads(tB_str)[tokenId.toString()]

aB_str = c_utils.java_to_json(AccountBalanceQuery().setAccountId(testId).execute(client).tokens.toString())
aB = json.loads(aB_str)[tokenId.toString()]

print('='*20)
print(f'Treasury Balance: {tB}')
print(f'Eric\'s Balance:{aB}')

Deposit Tokens from Test Account to Treasury: SUCCESS
Deposit Amount: 100
Treasury Balance: 10000
Eric's Balance:0


#### Query: Balance deposited into the instrument

In [30]:
query_result = (ContractCallQuery()
                .setGas(300000)
                .setContractId(contractId)
                .setFunction("getBalance")
                .setQueryPayment(Hbar(1))
                .execute(client))

message = query_result.getInt64(0)
print("Eric\'s Deposited Amount: ", message)

Eric's Deposited Amount:  100


#### Swap to Admin Account

In [31]:
OPERATOR_ID, OPERATOR_KEY, client = swap_to_user('admin')

#### Admin Payout to Eric

In [32]:
contractExecTx2 = (ContractExecuteTransaction()
                   .setContractId(contractId)
                   .setGas(3000000)
                   .setFunction("payout", ContractFunctionParameters()
                                .addAddress(testId.toSolidityAddress()))
                   .setMaxTransactionFee(Hbar(5))
                   .freezeWith(client))
contractExecSign2 = contractExecTx2.sign(testKey)
contractExecSubmit2 = contractExecTx2.execute(client)
contractExecRx2 = contractExecSubmit2.getReceipt(client)

print(f'Token Transfer from Treasury to Test Account: {contractExecRx2.status.toString()}')

tB_str = c_utils.java_to_json(AccountBalanceQuery().setAccountId(treasuryId).execute(client).tokens.toString())
tB = json.loads(tB_str)[tokenId.toString()]

aB_str = c_utils.java_to_json(AccountBalanceQuery().setAccountId(testId).execute(client).tokens.toString())
aB = json.loads(aB_str)[tokenId.toString()]

print('='*20)
print(f'Treasury Balance: {tB}')
print(f'Eric\'s Balance: {aB}')

Token Transfer from Treasury to Test Account: SUCCESS
Treasury Balance: 9800
Eric's Balance: 200
